<a href="https://colab.research.google.com/github/ZeinabAghahadi/Syllogistic-Commonsense-Reasoning/blob/main/Avicenna_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning Roberta for Syllogistic NLI with Avicenna Dataset





<a id='section01'></a>
### Importing Libraries



In [1]:
!pip install transformers==3.0.2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 30.8 MB/s 
     |████████████████████████████████| 3.0 MB 7.4 MB/s 
     |████████████████████████████████| 880 kB 50.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0caea7c462fa90d5f8485dd2d0354ce0f710890b40ed0d74cc40162bb98d624f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


<a id='section03'></a>
### Reading the Avicenna dataset



In [2]:
import pandas as pd
Train_url="https://raw.githubusercontent.com/ZeinabAghahadi/Syllogistic-Commonsense-Reasoning/main/Data/RoBERTa/reg_adv_train.csv"
train = pd.read_csv(Train_url, encoding= 'unicode_escape')


In [3]:
train.shape


(5119, 2)

In [4]:
train.head()


,Premises,label
0,Sodium plays a vital role in maintenance and r...,0
1,"Most people who deal with loss, really find it...",0
2,Insufficient income makes employees quit their...,0
3,Free radicals are released by foods that are f...,0
4,the construction of dams severely affects rive...,1


In [5]:
new_df = train[['Premises', 'label']]


In [6]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [7]:
class SyllogismData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Premises
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [8]:
train_size = 1

train_data=new_df.sample(frac=train_size,random_state=0)
train_data = train_data.reset_index(drop=True)

print("FULL train Dataset: {}".format(new_df.shape))

print("TRAIN Dataset: {}".format(train_data.shape))

training_set = SyllogismData(train_data, tokenizer, MAX_LEN)


FULL train Dataset: (5119, 2)
TRAIN Dataset: (5119, 2)


In [9]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)


<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`. 
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs. 
 - The data will be fed to the Roberta Language model as defined in the Avicenna dataset. 
 - Final layer outputs is what will be compared to the `Syllogistic relation` to determine the accuracy of models prediction. 
 


In [10]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [11]:
model = RobertaClass()
model.to(device)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, eleme

<a id='section05'></a>
### Fine Tuning the Model



In [12]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [13]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [14]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [15]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 5000 steps: 1.708037257194519
Training Accuracy per 5000 steps: 0.0


640it [03:53,  2.74it/s]


The Total Accuracy for Epoch 0: 68.21644852510256
Training Loss Epoch: 0.6120537128066644
Training Accuracy Epoch: 68.21644852510256


0it [00:00, ?it/s]

...

The Total Accuracy for Epoch 8: 99.33580777495605
Training Loss Epoch: 0.020490384277627526
Training Accuracy Epoch: 99.33580777495605


0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.07518977671861649
Training Accuracy per 5000 steps: 100.0


640it [03:54,  2.72it/s]

The Total Accuracy for Epoch 9: 99.14045712053135
Training Loss Epoch: 0.027359391917798348
Training Accuracy Epoch: 99.14045712053135


<a id='section06'></a>
### Validating the Model

The unseen data is the 20% of Avicenna.

In [16]:
Test_url = 'https://raw.githubusercontent.com/ZeinabAghahadi/Syllogistic-Commonsense-Reasoning/main/Data/RoBERTa/reg_test.csv'
test = pd.read_csv(Test_url, encoding= 'unicode_escape')
test.head()
test['label'].unique()
test.describe()
testnew_df = test[['Premises', 'label']]

test_size = 1
test_data=testnew_df

print("FULL test Dataset: {}".format(testnew_df.shape))
print("TEST Dataset: {}".format(test_data.shape))

testing_set = SyllogismData(test_data, tokenizer, MAX_LEN)


test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
testing_loader = DataLoader(testing_set, **test_params)

FULL test Dataset: (1200, 2)
TEST Dataset: (1200, 2)


In [17]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)
            #print(big_idx)


            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples

    return epoch_accu


In [18]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

300it [00:19, 15.78it/s]

Accuracy on test data = 88.17%
